In [1]:
!pip freeze > requirements.txt

In [2]:
!pip install dill

In [3]:
import pandas as pd
import dill
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression #Надо поменять модель, наверное
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, StandardScaler

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score, recall_score

from sklearn.impute import SimpleImputer

import sklearn.datasets

In [4]:
df = pd.read_csv('news.csv')

In [5]:
df.head(3)

,Unnamed: 0.1,Unnamed: 0,title,text,label
0,0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,1


In [6]:
df = df.loc[:, ~df.columns.str.contains('Unnamed')]
df.shape

(6335, 3)

In [7]:
df.describe()

,label
count,6335.000000
mean,0.500552
std,0.500039
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [8]:
df['label'].value_counts()

1    3171
0    3164
Name: label, dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   6335 non-null   object
 1   text    6335 non-null   object
 2   label   6335 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 148.6+ KB


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df, df['label'], test_size=0.33, random_state=42, stratify=df['label'])

X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [11]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.key] = X[self.key].astype(str)
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [12]:
title = Pipeline([
    ('imputer', TextImputer('title', ' ')),
    ('selector', ColumnSelector(key='title')),
    ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
])


text = Pipeline([
    ('imputer', TextImputer('text', ' ')),
    ('selector', ColumnSelector(key='text')),
    ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
])


feats = FeatureUnion([
    ('title', title),
    ('text', text),
])

In [16]:
pipeline = Pipeline([
    ('features', feats),
    ('classifier', SVC(kernel='linear', probability=True)),
])

pipeline.fit(X_test, y_test)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('title',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='title',
                                                                              value=' ')),
                                                                 ('selector',
                                                                  ColumnSelector(key='title')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(max_df=0.9,
                                                                                  min_df=10))])),
                                                ('text',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(k

In [17]:
with open("SVC_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)